In [ ]:
# Calculate elevation for each watershed from a 30m DEM and a 1km DEM

# Buffer watersheds to make sure there's enough data to generate 1km DEM
arcpy.analysis.PairwiseBuffer(
    in_features="NEON_Aquatic_Watershed",
    out_feature_class=r"C:\Users\User\Documents\UNR\NEON Project\Data\GIS\NEON_watersheds\buffer.gdb\NEON_watershed_buffer",
    buffer_distance_or_field="1.5 Kilometers",
    dissolve_option="NONE",
    dissolve_field=None,
    method="PLANAR",
    max_deviation="0 DecimalDegrees"
)

# Download and mosaic 30m DEM from USGS covering all watersheds
with arcpy.EnvManager(nodata="-999999"):
    arcpy.management.MosaicToNewRaster(
        input_rasters="USGS_1_n37w120_20211006.tif;USGS_1_n38w120_20210701.tif;USGS_1_n45w123_20220426.tif;USGS_1_n46w122_20211129.tif;USGS_1_n41w112_20221115.tif;USGS_1_n35w112_20211004.tif;USGS_1_n34w112_20230320.tif;USGS_1_n41w106_20230314.tif;USGS_1_n40w106_20230602.tif;USGS_1_n45w111_20230117.tif;USGS_1_n34w098_20211124.tif;USGS_1_n35w097_20191125.tif;USGS_1_n40w104_20230602.tif;USGS_1_n40w103_20211005.tif;USGS_1_n36w089_20220729.tif;USGS_1_n32w089_20220728.tif;USGS_1_n32w088_20230306.tif;USGS_1_n35w090_20221121.tif;USGS_1_n35w089_20221121.tif;USGS_1_n35w088_20220728.tif;USGS_1_n35w087_20220728.tif;USGS_1_n34w087_20220728.tif;USGS_1_n34w088_20200313.tif;USGS_1_n34w090_20221121.tif;USGS_1_n34w089_20220601.tif;USGS_1_n33w089_20220728.tif;USGS_1_n33w088_20220728.tif;USGS_1_n36w085_20220725.tif;USGS_1_n36w084_20220725.tif;USGS_1_n39w097_20230613.tif;USGS_1_n40w097_20230613.tif;USGS_1_n34w085_20230215.tif;USGS_1_n33w084_20230215.tif;USGS_1_n32w084_20230215.tif;USGS_1_n33w085_20230215.tif;USGS_1_n32w085_20230215.tif;USGS_1_n39w079_20211229.tif;USGS_1_n40w078_20230620.tif;USGS_1_n43w073_20230117.tif;USGS_1_n40w079_20230816.tif",
        output_location=r"C:\Users\User\Documents\UNR\NEON Project\Data\GIS\DEM\DEM_30m",
        raster_dataset_name_with_extension="DEM_20m.tif",
        coordinate_system_for_the_raster='GEOGCS["GCS_North_American_1983",DATUM["D_North_American_1983",SPHEROID["GRS_1980",6378137.0,298.257222101]],PRIMEM["Greenwich",0.0],UNIT["Degree",0.0174532925199433]]',
        pixel_type="32_BIT_FLOAT",
        cellsize=None,
        number_of_bands=1,
        mosaic_method="LAST",
        mosaic_colormap_mode="FIRST"
    )

# Mean elevation from 30m DEM
arcpy.sa.ZonalStatisticsAsTable(
    in_zone_data="NEON_Aquatic_Watershed",
    zone_field="SiteID",
    in_value_raster="DEM_30m.tif",
    out_table=r"C:\Users\User\Documents\UNR\NEON Project\Data\GIS\WaterIsotopesDatabase\tables.gdb\DEM30m",
    ignore_nodata="DATA",
    statistics_type="MEAN",
    process_as_multidimensional="CURRENT_SLICE",
    percentile_values=[90],
    percentile_interpolation_type="AUTO_DETECT",
    circular_calculation="ARITHMETIC",
    circular_wrap_value=360
)

# Reproject DEM to match isotope grid
with arcpy.EnvManager(outputCoordinateSystem='PROJCS["Unknown_based_on_Normal_Sphere_r_6370997_ellipsoid_Lambert_Azimuthal_Equal_Area",GEOGCS["GCS_unknown",DATUM["D_Unknown_based_on_Normal_Sphere_r_6370997_ellipsoid",SPHEROID["Normal_Sphere_r_6370997",6370997.0,0.0]],PRIMEM["Greenwich",0.0],UNIT["Degree",0.0174532925199433]],PROJECTION["Lambert_Azimuthal_Equal_Area"],PARAMETER["false_easting",0.0],PARAMETER["false_northing",0.0],PARAMETER["central_meridian",-100.0],PARAMETER["latitude_of_origin",45.0],UNIT["Meter",1.0]]', resamplingMethod="BILINEAR", snapRaster="d2h_01.tif", nodata="-999999", cellSize="d2h_01.tif"):
    arcpy.management.ProjectRaster(
        in_raster="DEM_30m.tif",
        out_raster=r"C:\Users\User\Documents\UNR\NEON Project\Data\GIS\DEM\DEM_1km\DEM_1km.tif",
        out_coor_system='PROJCS["Unknown_based_on_Normal_Sphere_r_6370997_ellipsoid_Lambert_Azimuthal_Equal_Area",GEOGCS["GCS_unknown",DATUM["D_Unknown_based_on_Normal_Sphere_r_6370997_ellipsoid",SPHEROID["Normal_Sphere_r_6370997",6370997.0,0.0]],PRIMEM["Greenwich",0.0],UNIT["Degree",0.0174532925199433]],PROJECTION["Lambert_Azimuthal_Equal_Area"],PARAMETER["false_easting",0.0],PARAMETER["false_northing",0.0],PARAMETER["central_meridian",-100.0],PARAMETER["latitude_of_origin",45.0],UNIT["Meter",1.0]]',
        resampling_type="BILINEAR",
        cell_size="1000 1000",
        geographic_transform=None,
        Registration_Point=None,
        in_coor_system='GEOGCS["GCS_North_American_1983",DATUM["D_North_American_1983",SPHEROID["GRS_1980",6378137.0,298.257222101]],PRIMEM["Greenwich",0.0],UNIT["Degree",0.0174532925199433]]',
        vertical="NO_VERTICAL"
    )
    
arcpy.sa.ZonalStatisticsAsTable(
    in_zone_data="NEON_Aquatic_Watershed",
    zone_field="SiteID",
    in_value_raster="DEM_1km.tif",
    out_table=r"C:\Users\User\Documents\UNR\NEON Project\Data\GIS\WaterIsotopesDatabase\tables.gdb\DEM1km",
    ignore_nodata="DATA",
    statistics_type="MEAN",
    process_as_multidimensional="CURRENT_SLICE",
    percentile_values=[90],
    percentile_interpolation_type="AUTO_DETECT",
    circular_calculation="ARITHMETIC",
    circular_wrap_value=360
)

In [4]:
from arcpy.sa import *

# Calculate precipitation-weighted mean elevation of each watershed
summer_p = (Raster("PRISM_ppt_30yr_normal_800mM4_04_asc.asc") +
           Raster("PRISM_ppt_30yr_normal_800mM4_05_asc.asc") +
           Raster("PRISM_ppt_30yr_normal_800mM4_06_asc.asc") +
           Raster("PRISM_ppt_30yr_normal_800mM4_07_asc.asc") +
           Raster("PRISM_ppt_30yr_normal_800mM4_08_asc.asc") +
           Raster("PRISM_ppt_30yr_normal_800mM4_09_asc.asc"))

summer_p.save(r"C:\Users\User\Documents\UNR\NEON Project\Data\GIS\PRISM\sum_ppt_summer.tif")

winter_p = (Raster("PRISM_ppt_30yr_normal_800mM4_01_asc.asc") +
           Raster("PRISM_ppt_30yr_normal_800mM4_02_asc.asc") +
           Raster("PRISM_ppt_30yr_normal_800mM4_03_asc.asc") +
           Raster("PRISM_ppt_30yr_normal_800mM4_10_asc.asc") +
           Raster("PRISM_ppt_30yr_normal_800mM4_11_asc.asc") +
           Raster("PRISM_ppt_30yr_normal_800mM4_12_asc.asc"))

winter_p.save(r"C:\Users\User\Documents\UNR\NEON Project\Data\GIS\PRISM\sum_ppt_winter.tif")

with arcpy.EnvManager(outputCoordinateSystem='GEOGCS["GCS_North_American_1983",DATUM["D_North_American_1983",SPHEROID["GRS_1980",6378137.0,298.257222101]],PRIMEM["Greenwich",0.0],UNIT["Degree",0.0174532925199433]]', snapRaster="DEM_30m.tif", nodata="-9999"):
    arcpy.management.Resample(
        in_raster="sum_ppt_summer.tif",
        out_raster=r"C:\Users\User\Documents\UNR\NEON Project\Data\GIS\PRISM\sum_ppt_summer_30m.tif",
        cell_size="2.77777777786987E-04 2.77777777786987E-04",
        resampling_type="NEAREST"
    )

with arcpy.EnvManager(outputCoordinateSystem='GEOGCS["GCS_North_American_1983",DATUM["D_North_American_1983",SPHEROID["GRS_1980",6378137.0,298.257222101]],PRIMEM["Greenwich",0.0],UNIT["Degree",0.0174532925199433]]', snapRaster="DEM_30m.tif", nodata="-9999"):
    arcpy.management.Resample(
        in_raster="sum_ppt_winter.tif",
        out_raster=r"C:\Users\User\Documents\UNR\NEON Project\Data\GIS\PRISM\sum_ppt_winter_30m.tif",
        cell_size="2.77777777786987E-04 2.77777777786987E-04",
        resampling_type="NEAREST"
    )




In [8]:
summer_p_x_elev = (Raster("sum_ppt_summer_30m.tif") * Raster("DEM_30m.tif"))
winter_p_x_elev = (Raster("sum_ppt_winter_30m.tif") * Raster("DEM_30m.tif"))

summer_p_x_elev.save(r"C:\Users\User\Documents\UNR\NEON Project\Data\GIS\PRISM\summer_p_x_elev.tif")
winter_p_x_elev.save(r"C:\Users\User\Documents\UNR\NEON Project\Data\GIS\PRISM\winter_p_x_elev.tif")



In [9]:
arcpy.sa.ZonalStatisticsAsTable(
    in_zone_data="NEON_Aquatic_Watershed",
    zone_field="SiteID",
    in_value_raster="summer_p_x_elev.tif",
    out_table=r"C:\Users\User\Documents\UNR\NEON Project\Data\GIS\PRISM\tables.gdb\sum_summer_p_x_elev",
    ignore_nodata="DATA",
    statistics_type="SUM",
    process_as_multidimensional="CURRENT_SLICE",
    percentile_values=[90],
    percentile_interpolation_type="AUTO_DETECT",
    circular_calculation="ARITHMETIC",
    circular_wrap_value=360
)

arcpy.sa.ZonalStatisticsAsTable(
    in_zone_data="NEON_Aquatic_Watershed",
    zone_field="SiteID",
    in_value_raster="winter_p_x_elev.tif",
    out_table=r"C:\Users\User\Documents\UNR\NEON Project\Data\GIS\PRISM\tables.gdb\sum_winter_p_x_elev",
    ignore_nodata="DATA",
    statistics_type="SUM",
    process_as_multidimensional="CURRENT_SLICE",
    percentile_values=[90],
    percentile_interpolation_type="AUTO_DETECT",
    circular_calculation="ARITHMETIC",
    circular_wrap_value=360
)

arcpy.sa.ZonalStatisticsAsTable(
    in_zone_data="NEON_Aquatic_Watershed",
    zone_field="SiteID",
    in_value_raster="sum_ppt_summer_30m.tif",
    out_table=r"C:\Users\User\Documents\UNR\NEON Project\Data\GIS\PRISM\tables.gdb\sum_summer_p",
    ignore_nodata="DATA",
    statistics_type="SUM",
    process_as_multidimensional="CURRENT_SLICE",
    percentile_values=[90],
    percentile_interpolation_type="AUTO_DETECT",
    circular_calculation="ARITHMETIC",
    circular_wrap_value=360
)

arcpy.sa.ZonalStatisticsAsTable(
    in_zone_data="NEON_Aquatic_Watershed",
    zone_field="SiteID",
    in_value_raster="sum_ppt_winter_30m.tif",
    out_table=r"C:\Users\User\Documents\UNR\NEON Project\Data\GIS\PRISM\tables.gdb\sum_winter_p",
    ignore_nodata="DATA",
    statistics_type="SUM",
    process_as_multidimensional="CURRENT_SLICE",
    percentile_values=[90],
    percentile_interpolation_type="AUTO_DETECT",
    circular_calculation="ARITHMETIC",
    circular_wrap_value=360
)


<geoprocessing server result object object at 0x0000022EB9B43CC0>

In [1]:
arcpy.sa.ZonalStatisticsAsTable(
    in_zone_data="NEON_Aquatic_Watershed",
    zone_field="SiteID",
    in_value_raster="PRISM_tmean_30yr_normal_800mM4_annual_asc.asc",
    out_table=r"C:\Users\User\Documents\UNR\NEON Project\Data\GIS\PRISM\tables.gdb\temperature",
    ignore_nodata="DATA",
    statistics_type="MEAN",
    process_as_multidimensional="CURRENT_SLICE",
    percentile_values=[90],
    percentile_interpolation_type="AUTO_DETECT",
    circular_calculation="ARITHMETIC",
    circular_wrap_value=360
)


<geoprocessing server result object object at 0x000001B22B975C60>

In [3]:
import pandas as pd

table = r"C:\Users\User\Documents\UNR\NEON Project\Data\GIS\PRISM\tables.gdb\sum_summer_p_x_elev"
columns = [f.name for f in arcpy.ListFields(table) if f.type!="Geometry"] 
summer_p_x_elev = pd.DataFrame(data=arcpy.da.SearchCursor(table, columns), columns=columns)

table = r"C:\Users\User\Documents\UNR\NEON Project\Data\GIS\PRISM\tables.gdb\sum_winter_p_x_elev"
columns = [f.name for f in arcpy.ListFields(table) if f.type!="Geometry"] 
winter_p_x_elev = pd.DataFrame(data=arcpy.da.SearchCursor(table, columns), columns=columns)

table = r"C:\Users\User\Documents\UNR\NEON Project\Data\GIS\PRISM\tables.gdb\sum_summer_p"
columns = [f.name for f in arcpy.ListFields(table) if f.type!="Geometry"] 
summer_p = pd.DataFrame(data=arcpy.da.SearchCursor(table, columns), columns=columns)

table = r"C:\Users\User\Documents\UNR\NEON Project\Data\GIS\PRISM\tables.gdb\sum_winter_p"
columns = [f.name for f in arcpy.ListFields(table) if f.type!="Geometry"] 
winter_p = pd.DataFrame(data=arcpy.da.SearchCursor(table, columns), columns=columns)

table = r"C:\Users\User\Documents\UNR\NEON Project\Data\GIS\PRISM\tables.gdb\temperature"
columns = [f.name for f in arcpy.ListFields(table) if f.type!="Geometry"] 
temperature = pd.DataFrame(data=arcpy.da.SearchCursor(table, columns), columns=columns)

wtd_elev = {}
for s in range(len(summer_p['SiteID'])):
    site = summer_p['SiteID'].iloc[s]
    wtd_elev[s] = (site,
                     summer_p_x_elev.loc[summer_p_x_elev['SiteID'] == site,'SUM'].iloc[0] / 
                     summer_p.loc[summer_p['SiteID'] == site, 'SUM'].iloc[0],
                     winter_p_x_elev.loc[winter_p_x_elev['SiteID'] == site,'SUM'].iloc[0] / 
                     winter_p.loc[winter_p['SiteID'] == site, 'SUM'].iloc[0],
                     temperature.loc[temperature['SiteID'] == site, 'MEAN'].iloc[0])
    
df = pd.DataFrame.from_dict(wtd_elev, orient='index', columns=['site', 'summer_wtd_elev', 'winter_wtd_elev', 'temperature'])

df.to_csv(r'C:\Users\User\Documents\UNR\NEON Project\Data\Compiled\wtd_elev.csv')

